<a href="https://colab.research.google.com/github/lisya/ML_in_business/blob/hw_final/3_Requests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import requests
import urllib.request
import json 
from pprint import pprint

In [2]:
test_data = {'CreditScore': 596,
            'Geography': 'Germany',
            'Gender': 'Male',
            'Age': 32,
            'Tenure': 3,
            'Balance': 96709.07,
            'NumOfProducts': 2,
            'HasCrCard': 0,
            'IsActiveMember': 0,
            'EstimatedSalary': 41788.37}

In [3]:
# формируем запрос
def send_json(x):
# На вход должен приходить словарь. Но если мы делаем Мы передаем на вход
# Series из нашего датафрейма то необходима дополнительня обработка.
# Идет прорерка является ли объект серией. Если да то переводиим в словарь.
# Теперь нужно изменить числовые типы numpy данных, т.к. эти данные не смогут
# сконвертироваться в json и будет ошибка


    if isinstance(x, pd.Series):
      x = x.to_dict()

      for key, value in x.items():
        if isinstance(value, (np.int_, np.intc, np.intp, np.int8,
                          np.int16, np.int32, np.int64, np.uint8,
                          np.uint16, np.uint32, np.uint64)):
          x[key] = int(value)
        elif isinstance(value, (np.float_, np.float16, np.float32, np.float64)):
          x[key] = float(value)


    myurl = 'http://bd6441f246ae.ngrok.io/'+'predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=x, headers=headers)
    return float(response.json()['predictions'])

# обращение к серверу с запросом из одного набора (его построили руками выше - test_data)
if __name__ == '__main__':
    response = send_json(test_data)
    print('Данные:\n')
    pprint(test_data)
    print('\n Предсказание:',response)

Данные:

{'Age': 32,
 'Balance': 96709.07,
 'CreditScore': 596,
 'EstimatedSalary': 41788.37,
 'Gender': 'Male',
 'Geography': 'Germany',
 'HasCrCard': 0,
 'IsActiveMember': 0,
 'NumOfProducts': 2,
 'Tenure': 3}

 Предсказание: 0.03430735


Сделаем обработку массы запросов:

Загрузим данные X_test

In [4]:
path = '/content/drive/MyDrive/Notebooks/ML_in_business/final_project/'

In [5]:
X_test = pd.read_csv(path+"X_test.csv")
y_test = pd.read_csv(path+"y_test.csv")

In [6]:
from sklearn.metrics import roc_auc_score,roc_curve
from urllib import request, parse

In [7]:
response = send_json(X_test.iloc[0,:])
response

0.03430735

In [9]:
X_test.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,596,Germany,Male,32,3,96709.07,2,0,0,41788.37
1,623,France,Male,43,1,0.00,2,1,1,146379.30
2,601,Spain,Female,44,4,0.00,2,1,0,58561.31
3,506,Germany,Male,59,8,119152.10,2,1,1,170679.74
4,560,Spain,Female,27,7,124995.98,1,1,1,114669.79


Сделаем N запросов и оценим время

In [10]:
N = 50

In [11]:
%%time
predictions = X_test.iloc[:N,:].apply(lambda x: send_json(x),1)

CPU times: user 141 ms, sys: 14.1 ms, total: 156 ms
Wall time: 15.5 s


In [12]:
predictions.values[:5]

array([0.03430735, 0.03184487, 0.10528768, 0.14373377, 0.0716122 ])

Посчитаем метрику roc-auc

In [13]:
y_test.head()

,Exited
0,0
1,0
2,0
3,0
4,0


In [14]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

0.9035087719298246